In [1]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 2.8 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pathlib pydicom matplotlib tqdm opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision torchmetrics pytorch-lightning

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U tensorboardX

  Attempting uninstall: tensorboardX
    Found existing installation: tensorboardX 2.6.2.2
    Uninstalling tensorboardX-2.6.2.2:
      Successfully uninstalled tensorboardX-2.6.2.2
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import shutil
import pandas as pd

# Paths
root_folder = "C:/Users/n.kulangareth/Downloads/rsna-pneumonia-detection-challenge/stage_2_train_images"
process_folder = "C:/Users/n.kulangareth/Downloads/500_samples"

# Read the CSV file (update with actual file name)
csv_path = "C:/Users/n.kulangareth/Downloads/500_samples/stage_2_train_labels.csv"
labels = pd.read_csv(csv_path)
labels.head(5)

# Ensure process folder exists
os.makedirs(process_folder, exist_ok=True)

# Check if 'patientId' column exists
if 'patientId' in labels.columns:
    for pid in labels['patientId']:
        # Look for files named as the patientId with any extension
        for file_name in os.listdir(root_folder):
            name, ext = os.path.splitext(file_name)
            if name == str(pid):
                src = os.path.join(root_folder, file_name)
                dst = os.path.join(process_folder, file_name)
                shutil.copy2(src, dst)
                break  # Stop checking once we find a matching file
    print("Files copied successfully.")
else:
    print("Column 'patientId' not found in CSV.")


Files copied successfully.


In [3]:
from pathlib import Path
import pydicom
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torchvision
from torchvision import transforms
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [4]:

# Read the CSV file (update with actual file name)
csv_path = "C:/Users/n.kulangareth/Downloads/all_samples/stage_2_train_labels.csv"
labels = pd.read_csv(csv_path)
labels = labels.drop_duplicates("patientId")
from tqdm import tqdm
sums = 0
sums_squared = 0

ROOT_PATH = Path("C:/Users/n.kulangareth/Downloads/all_samples/stage_2_train_images")
SAVE_PATH = Path("C:/Users/n.kulangareth/Downloads/all_samples/Processed/")
for c, patient_id in enumerate(tqdm(labels.patientId.astype(str))):
    dcm_path = ROOT_PATH / patient_id
    dcm_path = dcm_path.with_suffix(".dcm")


   
    # Read the DICOM file with pydicom and standardize the array
    dcm = pydicom.dcmread(dcm_path).pixel_array / 255  # Use dcmread instead of read_file
       
    # Resize the image; 1024x1024 is large, so we resize to 224x224
    dcm_array = cv2.resize(dcm, (224, 224)).astype(np.float16)
   
    # Retrieve the corresponding label
    label = labels.Target.iloc[c]
   
    # 4/5 train split, 1/5 val split
    train_or_val = "train" if c < 24000 else "val"
       
    # Define save path and create directories if necessary
    current_save_path = SAVE_PATH / train_or_val / str(label)
    current_save_path.mkdir(parents=True, exist_ok=True)
   
    # Save the array in the corresponding directory
    np.save(current_save_path / patient_id, dcm_array)
   
    # Normalize sum of image for statistics calculation
    normalizer = dcm_array.shape[0] * dcm_array.shape[1]
   
    if train_or_val == "train":  # Only use train data to compute dataset statistics
        sums += np.sum(dcm_array) / normalizer
        sums_squared += (np.power(dcm_array, 2).sum()) / normalizer



mean = sums / 24000
std = np.sqrt(sums_squared / 24000 - (mean**2))


print(f"Mean of Dataset: {mean}, STD: {std}")

100%|████████████████████████████████████████████████████████████████████████████| 26684/26684 [08:02<00:00, 55.33it/s]


Mean of Dataset: 0.4903962485384803, STD: 0.24795070634161256


In [5]:



def load_file(path):
    return np.load(path).astype(np.float32)

train_transforms = transforms.Compose([
                                    transforms.ToTensor(),  # Convert numpy array to tensor
                                    transforms.Normalize(0.4903, 0.2479),  # Use mean and std from preprocessing notebook
                                    transforms.RandomAffine( # Data Augmentation
                                        degrees=(-5, 5), translate=(0, 0.05), scale=(0.9, 1.1)),
                                        transforms.RandomResizedCrop((224, 224), scale=(0.35, 1))

])

val_transforms = transforms.Compose([
                                    transforms.ToTensor(),  # Convert numpy array to tensor
                                    transforms.Normalize([0.4903], [0.2479]),  # Use mean and std from preprocessing notebook
])








In [6]:
train_dataset = torchvision.datasets.DatasetFolder(
    "C:/Users/n.kulangareth/Downloads/all_samples/Processed/train",
    loader=load_file, extensions="npy", transform=train_transforms)

val_dataset = torchvision.datasets.DatasetFolder(
    "C:/Users/n.kulangareth/Downloads/all_samples/Processed/val",
    loader=load_file, extensions="npy", transform=val_transforms)



In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(9, 9))
for i in range(2):
    for j in range(2):
        random_index = np.random.randint(0, 200)
        x_ray, label = train_dataset[random_index]
        axis[i][j].imshow(x_ray[0], cmap="bone")
        axis[i][j].set_title(f"Label:{label}")

In [7]:

batch_size = 8#TODO
num_workers = 0# TODO

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

print(f"There are {len(train_dataset)} train images and {len(val_dataset)} val images")

import torch
import torchvision
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

class PneumoniaModel(pl.LightningModule):
    def __init__(self, weight=1):
        super().__init__()

        self.model = torchvision.models.resnet18()
        # Modify input and output layers
        self.model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = torch.nn.Linear(in_features=512, out_features=1)

        self.loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weight]))
        self.train_acc = torchmetrics.classification.BinaryAccuracy()
        self.val_acc = torchmetrics.classification.BinaryAccuracy()

    def forward(self, data):
        return self.model(data)

    def training_step(self, batch, batch_idx):
        x_ray, label = batch
        label = label.float()
        pred = self(x_ray)[:, 0]
        loss = self.loss_fn(pred, label)

        self.log("Train Loss", loss, on_step=True, on_epoch=False)
        self.log("Step Train Acc", self.train_acc(torch.sigmoid(pred), label.int()), on_step=True, on_epoch=False)

        return loss

    def on_train_epoch_end(self):
        self.log("Train Acc", self.train_acc.compute(), prog_bar=True)
        self.train_acc.reset()

    def validation_step(self, batch, batch_idx):
        x_ray, label = batch
        label = label.float()
        pred = self(x_ray)[:, 0]
        loss = self.loss_fn(pred, label)

        self.log("Val Loss", loss, on_step=True, on_epoch=False)
        self.log("Step Val Acc", self.val_acc(torch.sigmoid(pred), label.int()), on_step=True, on_epoch=False)

        return loss

    def on_validation_epoch_end(self):
        self.log("Val Acc", self.val_acc.compute(), prog_bar=True)
        self.val_acc.reset()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)


There are 24000 train images and 2684 val images


In [8]:
# Instantiate model
model = PneumoniaModel()

# Checkpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='Val Acc',
    save_top_k=10,
    mode='max'
)

# Trainer
trainer = pl.Trainer(
    accelerator="cpu",  # Change to "gpu" if needed
    devices=1,
    logger=TensorBoardLogger(save_dir="./logs"),
    log_every_n_steps=1,
    callbacks=[checkpoint_callback],
    max_epochs=35
)

# Train the model
trainer.fit(model, train_loader, val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ResNet            | 11.2 M | train
1 | loss_fn   | BCEWithLogitsLoss | 0      | train
2 | train_acc | BinaryAccuracy    | 0      | train
3 | val_acc   | BinaryAccuracy    | 0      | train
--------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.683    Total estimated model params size (MB)
71        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

C:\Users\n.kulangareth\AppData\Local\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
C:\Users\n.kulangareth\AppData\Local\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=35` reached.


In [9]:

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [10]:
from tqdm import tqdm
import torch
from torchmetrics.classification import Accuracy, Precision, Recall, ConfusionMatrix

model.eval()
model.to(device)

preds = []
labels = []

# Use DataLoader and tqdm with progress bar
with torch.no_grad():
    for data, label in tqdm(val_loader, desc="Evaluating", unit="batch"):
        data = data.to(device).float()
        label = label.to(device).int()

        output = torch.sigmoid(model(data)).squeeze()
        preds.extend(output.cpu())
        labels.extend(label.cpu())

# Convert to tensors
preds = torch.stack(preds)
labels = torch.stack(labels)

# Thresholding
threshold = 0.5
pred_labels = (preds > threshold).int()

# Metrics
acc = Accuracy(task="binary")(pred_labels, labels)
precision = Precision(task="binary")(pred_labels, labels)
recall = Recall(task="binary")(pred_labels, labels)
cm = ConfusionMatrix(task="binary")(pred_labels, labels)

# Lower threshold example
pred_labels_thresh = (preds > 0.25).int()
cm_thresh = ConfusionMatrix(task="binary")(pred_labels_thresh, labels)

# Print metrics
print(f"Val Accuracy: {acc.item():.4f}")
print(f"Val Precision: {precision.item():.4f}")
print(f"Val Recall: {recall.item():.4f}")
print(f"Confusion Matrix (0.5 threshold):\n{cm}")
print(f"Confusion Matrix (0.25 threshold):\n{cm_thresh}")


Evaluating: 100%|█████████████████████████████████████████████████████████████████| 336/336 [00:43<00:00,  7.76batch/s]


Val Accuracy: 0.8454
Val Precision: 0.6877
Val Recall: 0.5752
Confusion Matrix (0.5 threshold):
tensor([[1921,  158],
        [ 257,  348]])
Confusion Matrix (0.25 threshold):
tensor([[1566,  513],
        [ 105,  500]])


In [ ]:
#started at 11:30 Sep 1 2025

In [1]:
class Dog:
    # Constructor: Initializes an object when created
    def __init__(self, name, breed, age):
        self.name = name  # instance variable
        self.breed = breed
        self.age = age

    # Method: A function defined inside a class
    def bark(self):
        print(f"{self.name} says Woof!")
    
    def eat(self):
        print(f"{self.name} is eating.")



In [ ]:
# Creating an object of the class
my_dog = Dog("Buddy", "Golden Retriever", 3)

# Accessing properties
print(my_dog.name)  # Outputs: Buddy

# Calling methods
my_dog.bark()  # Outputs: Buddy says Woof!
my_dog.eat()   # Outputs: Buddy is eating.
